In [2]:
!pip install gnews selenium webdriver-manager bs4


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=315098108bbb583a3a5f92e3253b513ff8a10fe9549809547f4db2f6b1af0956
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [3]:
import json
import time
from gnews import GNews
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


In [4]:
# Установка Chrome и ChromeDriver
!wget -q -O chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i chrome.deb || apt-get -fy install

# Установка Python-библиотек
!pip install selenium webdriver-manager gnews beautifulsoup4


Selecting previously unselected package google-chrome-stable.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack chrome.deb ...
Unpacking google-chrome-stable (138.0.7204.168-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package libvulkan1 is not installed.

dpkg: error processing package google-chrome-stable (--install):
 dependency problems - leaving unconfigured
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Errors were encountered while processing:
 google-chrome-stable
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  libvulkan1 mesa-vulkan-drivers
0 upgraded, 2 newly installed, 0 to remov

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
gnews = GNews(language='en', country='US', max_results=100)
queries = [
    "AI in healthcare",
    "artificial intelligence in medicine",
    "machine learning in healthcare",
    "AI diagnostics",
    "AI in medical imaging",
    "AI in hospitals"
]

seen_urls = set()
articles = []
MIN_WORDS = 50
MAX_ARTICLES = 100

def extract_text_with_selenium(url):
    try:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        paragraphs = soup.find_all("p")
        text = " ".join(p.get_text() for p in paragraphs)
        return text.strip()
    except Exception as e:
        print(f"⚠️ Ошибка парсинга: {e}")
        return None

for query in queries:
    results = gnews.get_news(query)
    print(f"🔍 {query} — {len(results)} найдено")
    for result in results:
        url = result.get("url")
        title = result.get("title", "No title")

        if not url or url in seen_urls:
            continue

        text = extract_text_with_selenium(url)
        if text and len(text.split()) >= MIN_WORDS:
            articles.append({
                "title": title,
                "url": url,
                "text": text
            })
            seen_urls.add(url)
            print(f"[+] {title[:60]} ({len(text.split())} слов)")

        if len(articles) >= MAX_ARTICLES:
            break
    if len(articles) >= MAX_ARTICLES:
        break



In [6]:
with open("ai_healthcare_full_articles.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)